## Day2 （環境から計画を立てる）

ここでは動的計画法(Dynamic　Programing: DP)を扱う。


In [1]:
import random
import sys
sys.path.append('../../baby-steps-of-rl-ja/DP/')

from environment import Environment

## Day1おさらい

- 強化学習はマルコフ決定過程を仮定し、その元で報酬の総和を最大化する方策をもつエージェントをもとめる問題
    - マルコフ決定過程は以下の(S, A, T, R)の組である
        - S: 状態全体の集合
        - A: アクション全体の集合
        - T: S×A×S -> [0, 1] 状態とアクションから次の状態への遷移確率を返す関数
        - R: S×A×S -> 実数全体 状態, アクション, 遷移状態の状態から報酬を返す関数


- Environmentクラス
    - 迷路環境クラス
    - State, Actionを内部に持つ
    - Environment.step()メソッドにアクションを渡すことで、内部で遷移確率T(s'| s, a)を計算し、Tに応じた(状態遷移/rewardの計算/終了判定)をする。
    
- Agentクラス
    - Environmentオブジェクトを持ち、stateに応じたアクションを返すpolicyメソッドを内部で持つ。


- 迷路に対し、ランダムでアクションを提案するpolicyを持つAgentを作成し、ゴール/禁止地点にたどり着くまでのrewardを計算してみる

In [2]:
# random Agent
class Agent():
    # initで入力した環境のactionをランダムに出力する。

    def __init__(self, env):
        self.actions = env.actions

    def policy(self, state):
        return random.choice(self.actions)

In [3]:
# Make grid environment.
grid = [
    [0, 0, 0, 1],
    [0, 9, 0, -1],
    [0, 0, 0, 0]
]
env = Environment(grid)
agent = Agent(env)

# Try 10 game.
for i in range(10):
    # Initialize position of agent.
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = agent.policy(state)
        next_state, reward, done = env.step(action)
        total_reward += reward
        state = next_state

    print("Episode {}: Agent gets {} reward.".format(i, total_reward))

Episode 0: Agent gets -2.12 reward.
Episode 1: Agent gets 0.56 reward.
Episode 2: Agent gets -2.3600000000000003 reward.
Episode 3: Agent gets -0.6000000000000008 reward.
Episode 4: Agent gets -0.16000000000000036 reward.
Episode 5: Agent gets -2.12 reward.
Episode 6: Agent gets 0.8 reward.
Episode 7: Agent gets -3.9200000000000017 reward.
Episode 8: Agent gets -0.28000000000000047 reward.
Episode 9: Agent gets 0.8 reward.


# Bellman方程式
- 価値評価関数を網羅的に、厳密に計算する例
- Value baseのiteration実装
- Policy baseのiteration実装

- policy baseのアクションは確率1/0しかないのか?（実装はそうなっている）　policyで複数アクションが等確率になった場合の計算方法は？

## Bellman方程式の計算(決定的)
- 価値関数の再帰表現を用いた価値計算
- 状態数が多くない & エピソードが長くない場合に計算可能

In [4]:
# happy endゲーム(仮)
# upかdownを繰り返し、5回行動したら終了。終了時upがHAPPY_END_BORDER以上なら"happy_end", そうでなければ"bad_end"
# MOVE


def V(s, gamma=0.99):
    """ 価値関数(再帰)
    次の状態の(遷移確率 * 価値)のmaxのみを計算に使用
    """
    V = R(s) + gamma * max_V_on_next_state(s)
    return V


def R(s):
    """ 報酬関数
    終了状態のみ±1, それ以外0
    """
    if s == "happy_end":
        return 1
    elif s == "bad_end":
        return -1
    else:
        return 0


def max_V_on_next_state(s):
    """ 
    全アクション, 全遷移先状態の(遷移確率 * 価値)のmaxの値を返す
    """
    # If game end, expected value is 0.
    if s in ["happy_end", "bad_end"]:
        return 0

    actions = ["up", "down"]
    values = []
    for a in actions:
        transition_probs = transit_func(s, a)
        v = 0
        for next_state in transition_probs:
            prob = transition_probs[next_state]
            v += prob * V(next_state)
        values.append(v)
    return max(values)


def transit_func(s, a):
    """ T(s'| s, a): 状態とアクション("up", "down")を受け取り、遷移確率を返す
    Make next state by adding action str to state.
    ex: (s = 'state', a = 'up') => 'state_up'
        (s = 'state_up', a = 'down') => 'state_up_down'
    """

    actions = s.split("_")[1:]
    LIMIT_GAME_COUNT = 5
    HAPPY_END_BORDER = 4
    MOVE_PROB = 0.9

    def next_state(state, action):
        return "_".join([state, action])

    # ゲーム終了, 確率1で終了結果　に遷移
    if len(actions) == LIMIT_GAME_COUNT:
        up_count = sum([1 if a == "up" else 0 for a in actions])
        state = "happy_end" if up_count >= HAPPY_END_BORDER else "bad_end"
        prob = 1.0
        return {state: prob}
    # ゲーム続行
    else:
        opposite = "up" if a == "down" else "down"
        return {
            next_state(s, a): MOVE_PROB,
            next_state(s, opposite): 1 - MOVE_PROB
        }


In [5]:
# 状態価値の算出
print(V("state"))
print(V("state_up_up"))
print(V("state_down_down"))

0.7880942034605892
0.9068026334400001
-0.96059601


## Bellman方程式の計算(動的計画法近似)

## planner
- 動的計画法による価値関数近似クラス
- Plannerクラスをケ継承してValueIterationとPolicyIterationクラスを実装

In [6]:
class Planner():

    def __init__(self, env):
        self.env = env
        self.log = []

    def initialize(self):
        self.env.reset()
        self.log = []

    def plan(self, gamma=0.9, threshold=0.0001):
        raise Exception("Planner have to implements plan method.")

    def transitions_at(self, state, action):
        transition_probs = self.env.transit_func(state, action)
        for next_state in transition_probs:
            prob = transition_probs[next_state]
            reward, _ = self.env.reward_func(next_state)
            yield prob, next_state, reward

    def dict_to_grid(self, state_reward_dict):
        grid = []
        for i in range(self.env.row_length):
            row = [0] * self.env.column_length
            grid.append(row)
        for s in state_reward_dict:
            grid[s.row][s.column] = state_reward_dict[s]

        return grid

## Value Iteration

In [7]:
class ValueIterationPlanner(Planner):

    def __init__(self, env):
        super().__init__(env)

    def plan(self, gamma=0.9, threshold=0.0001):
        self.initialize()
        actions = self.env.actions
        V = {}
        for s in self.env.states:
            # Initialize each state's expected reward.
            V[s] = 0

        while True:
            delta = 0
            self.log.append(self.dict_to_grid(V))
            for s in V:
                if not self.env.can_action_at(s):
                    continue
                expected_rewards = []
                for a in actions:
                    r = 0
                    for prob, next_state, reward in self.transitions_at(s, a):
                        r += prob * (reward + gamma * V[next_state])
                    expected_rewards.append(r)
                max_reward = max(expected_rewards)
                delta = max(delta, abs(max_reward - V[s]))
                V[s] = max_reward

            if delta < threshold:
                break

        V_grid = self.dict_to_grid(V)
        return V_grid

## Policy Iteration 

In [8]:
class PolicyIterationPlanner(Planner):

    def __init__(self, env):
        super().__init__(env)
        self.policy = {}

    def initialize(self):
        super().initialize()
        self.policy = {}
        actions = self.env.actions
        states = self.env.states
        for s in states:
            self.policy[s] = {}
            for a in actions:
                # Initialize policy.
                # At first, each action is taken uniformly.
                self.policy[s][a] = 1 / len(actions)

    def estimate_by_policy(self, gamma, threshold):
        V = {}
        for s in self.env.states:
            # Initialize each state's expected reward.
            V[s] = 0

        while True:
            delta = 0
            for s in V:
                expected_rewards = []
                for a in self.policy[s]:
                    action_prob = self.policy[s][a]
                    r = 0
                    for prob, next_state, reward in self.transitions_at(s, a):
                        r += action_prob * prob * \
                             (reward + gamma * V[next_state])
                    expected_rewards.append(r)
                value = sum(expected_rewards)
                delta = max(delta, abs(value - V[s]))
                V[s] = value
            if delta < threshold:
                break

        return V

    def plan(self, gamma=0.9, threshold=0.0001):
        self.initialize()
        states = self.env.states
        actions = self.env.actions

        def take_max_action(action_value_dict):
            return max(action_value_dict, key=action_value_dict.get)

        while True:
            update_stable = True
            # Estimate expected rewards under current policy.
            V = self.estimate_by_policy(gamma, threshold)
            self.log.append(self.dict_to_grid(V))

            for s in states:
                # Get an action following to the current policy.
                policy_action = take_max_action(self.policy[s])

                # Compare with other actions.
                action_rewards = {}
                for a in actions:
                    r = 0
                    for prob, next_state, reward in self.transitions_at(s, a):
                        r += prob * (reward + gamma * V[next_state])
                    action_rewards[a] = r
                best_action = take_max_action(action_rewards)
                if policy_action != best_action:
                    update_stable = False

                # Update policy (set best_action prob=1, otherwise=0 (greedy))
                for a in self.policy[s]:
                    prob = 1 if a == best_action else 0
                    self.policy[s][a] = prob

            if update_stable:
                # If policy isn't updated, stop iteration
                break

        # Turn dictionary to grid
        V_grid = self.dict_to_grid(V)
        return V_grid


# TODO
- policy/value iterationの証明
1. Bellman方程式の作用素表現 （value base, policy base）
1. 関数空間V上一様ノルムを入れた際の縮小写像証明 （maxとminを計算して上としたから抑える）
1. {T^nv}がcauchy列である証明（三角不等式と縮小写像）
1. （存在性）(距離空間の完備性より、{T^nv}は収束列で収束先v* :=lim_{n \to \infty}{T^nv}が存在)
1. Tv* =v*  の証明（「Tが縮小写像 => リプシッツ連続 => 連続関数」よりlimitの順状交換可能、ε-nから直接、など）
1. 任意の初期値v_0に対し、v* :=lim_{n \to \infty}{T^nv_0}の証明（2.と5.を複数回使用することで距離が0になる）
1. (一意性)一意性の証明もやる（v* 1, v* 2の距離をとって、Tで一回飛ばすとv* 1=v* 2以外矛盾）

上記より (value/policy)iterationによる近似が正当化される

関数空間Vの一様ノルムによる完備性は課題...